# Description requests

## Setup

In [1]:
# Import modules
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


In [2]:
# API connexion
client_id = 'cec8979c027344f98b471a991aa415ad'
client_secret = 'ad69bcba55b349f98c1344b006c708bd'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


## Get playlists description

In [ ]:
# Load data
df = pd.read_csv('df_final.csv')

# Get a list of the unique playlist names
playlist_names = df['playlist_name'].unique()
print(playlist_names)


In [ ]:
# API scraping
playlist_names = playlist_names.tolist()

playlists = []

for name in playlist_names:
    print(f"Processing playlist: {name}")

    playlist_id = "N/A"
    playlist_description = "N/A"

    try:
        id_results = sp.search(q=name, type='playlist', limit=1)

        # Check if any playlist is found
        if id_results['playlists']['items']:
            playlist_id = id_results['playlists']['items'][0]['id']
            playlists_results = sp.playlist(playlist_id)
            playlist_description = playlists_results.get('description', "N/A")
    
    except Exception as e:
        print(f"Error processing playlist {name}: {e}")

    # Append the playlist entry (even if API fails)
    playlists.append({
        "playlist_id": playlist_id, 
        "playlist_name": name,
        "playlist_description": playlist_description
    })

# Create dataframe from playlists
df_playlists = pd.DataFrame(playlists, columns=["playlist_id", "playlist_name", "playlist_description"])
df_playlists.to_excel("playlists_descriptions.xlsx", index=False)

print(df_playlists)


## Merge back with og data

In [2]:
# Load data
df_desc = pd.read_excel('playlists_descriptions.xlsx')
df  = pd.read_csv('df_final.csv')
print(len(df))


19153


In [3]:
# Drop the unnecessary columns
df_desc_2 = df_desc.drop(columns=['playlist_id', 'playlist_description'])
print(df_desc_2)


                                         playlist_name editorial_type type_2  \
0    Rap Fr | Rap Francais 2024 | Hits Rap |  SDM, ...          genre    NaN   
1    Rap Belge 2024 | Kobo, Damso, Caballero & Jean...          genre    NaN   
2    Musculation | MOTIVATION RAP 2024 Sport Rap | ...           mood  genre   
3                                             #NUITVIE           mood    NaN   
4    Cool Jazz (Celeste, Samara Joy, Melody Gardot,...          genre   mood   
..                                                 ...            ...    ...   
117                                        Futurs Hits          trend    NaN   
118                                      Shisha Lounge           mood    NaN   
119  Hits de l''et'e 90 2000, tubes annees 90, 2000...           mood    NaN   
120                                            Le Flow          genre    NaN   
121                                          Gym Tonic           mood    NaN   

    comments  
0        NaN  
1        

In [4]:
# Merge dataframes based on playlist name
df_final = pd.merge(df, df_desc_2, on='playlist_name', how='left')
df_final.to_csv('df_final_bis.csv', index=False)

print(df_final)


       Unnamed: 0                           playlist_id  \
0               1  11e84480-ad44-deb2-8ac8-a0369fe50396   
1               2  11e84480-ad44-deb2-8ac8-a0369fe50396   
2               3  11e84480-ad44-deb2-8ac8-a0369fe50396   
3               4  11e84480-ad44-deb2-8ac8-a0369fe50396   
4               5  11e84480-ad44-deb2-8ac8-a0369fe50396   
...           ...                                   ...   
19148       19149  f5de1012-b6d6-11e8-8a3a-525400009efb   
19149       19150  f5de1012-b6d6-11e8-8a3a-525400009efb   
19150       19151  f5de1012-b6d6-11e8-8a3a-525400009efb   
19151       19152  f5de1012-b6d6-11e8-8a3a-525400009efb   
19152       19153  f5de1012-b6d6-11e8-8a3a-525400009efb   

                                           playlist_name collection_date  \
0      Rap Fr | Rap Francais 2024 | Hits Rap |  SDM, ...      2021-10-01   
1      Rap Fr | Rap Francais 2024 | Hits Rap |  SDM, ...      2021-10-08   
2      Rap Fr | Rap Francais 2024 | Hits Rap |  SDM, ...      2